Importing Library Modules

In [1]:
import os
import numpy as np
import pandas as pd
import neattext.functions as nfx

Importing DataSet

In [2]:
# Current directory
print(os.getcwd())

D:\robolab\NLP_setv\MulticlassEmotionDetection


In [3]:
df = pd.read_csv(".\Dataset\emotion-dataset.csv")

In [4]:
df.head(3)

,Emotion,Text
0,neutral,Why ?
1,joy,Sage Act upgrade on my to do list for tommorow.
2,sadness,ON THE WAY TO MY HOMEGIRL BABY FUNERAL!!! MAN ...


Data Preprocessing

In [5]:
df["Emotion"].value_counts()

joy         11045
sadness      6722
fear         5410
anger        4297
surprise     4062
neutral      2254
disgust       856
shame         146
Name: Emotion, dtype: int64

In [6]:
dir(nfx)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [7]:
df['Text'][4]

"@Iluvmiasantos ugh babe.. hugggzzz for u .!  babe naamazed nga ako e babe e, despite nega's mas pinaramdam at fil ko ang "

In [8]:
# Removing user name/ Tag from the dataset
df['Clean_Text'] = df['Text'].apply(nfx.remove_userhandles)
df['Clean_Text'][4]

"  ugh babe.. hugggzzz for u .!  babe naamazed nga ako e babe e, despite nega's mas pinaramdam at fil ko ang "

In [9]:
# Removing Stop Words
df['Clean_Text'] = df['Clean_Text'].apply(nfx.remove_stopwords)

In [10]:
# Removing Punctuations
df.replace("[^a-zA-Z]"," ",regex=True, inplace =True)

In [11]:
df['lower'] = df['Clean_Text'].str.lower()
df['lower'][2]
df['Wordlist'] = df['Clean_Text'].str.lower()

In [12]:
# Preprocessed DataSet
df[['Emotion','Clean_Text']].head(4)

,Emotion,Clean_Text
0,neutral,
1,joy,Sage Act upgrade list tommorow
2,sadness,WAY HOMEGIRL BABY FUNERAL MAN HATE FUNERALS...
3,joy,eye true hazel eye and brilliant Regular f...


In [13]:
df[['Emotion','lower']].head(4)

,Emotion,lower
0,neutral,
1,joy,sage act upgrade list tommorow
2,sadness,way homegirl baby funeral man hate funerals...
3,joy,eye true hazel eye and brilliant regular f...


Importing machine learning packages

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [15]:
df['lower'][1]

'sage act upgrade list tommorow '

In [16]:
a = lemmatizer.lemmatize('cries')
print(a)

cry


In [17]:
print(len(df.index))

34792


In [18]:
for i in range(1,len(df.index)):
    words = nltk.word_tokenize(df['lower'][i])
    words = [stemmer.stem(word) for word in words]
    sentences = ' '.join(words)
    df['Wordlist'][i] = sentences

In [19]:
print(df['Wordlist'][28])

lose someth go fat burger dinner dad


In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [21]:
# Assigning X and Y from Dataset
Xfeatures = df['Wordlist']
ylabels = df['Emotion']

In [22]:
# Spliting Dataset
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=42)

In [23]:
# Bag of words
#countvector = CountVectorizer(ngram_range=(1,1))
countvector = CountVectorizer(ngram_range=(1,2))

In [24]:
trainDataset = countvector.fit_transform(x_train)

In [25]:
#trainDataset.toarray()

In [26]:
# Training the Logistic Regression model
logreg = LogisticRegression(solver='saga', max_iter=1000)
logreg.fit(trainDataset, y_train)

LogisticRegression(max_iter=1000, solver='saga')

In [27]:
# Testing the model
testDataset = countvector.transform(x_test)
y_pred = logreg.predict(testDataset)

In [28]:
# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6418854186625791


Trials

LogisticRegression(solver='saga', max_iter=1000) ........................... Accuracy: 0.6205211726384365
LogisticRegression(solver='sag', max_iter=1000) ............................ Accuracy: 0.6209043878137575
LogisticRegression(solver='newton-cg', max_iter=1000) ...................... Accuracy: 0.6209043878137575
LogisticRegression(solver='liblinear', max_iter=1000) ...................... Accuracy: 0.6253113623299482


gave higher Accuracy but result with custom text was bad
countvector = CountVectorizer(ngram_range=(1,2)) + LogisticRegression(solver='liblinear', max_iter=1000)................ Accuracy: 0.6403525579612953
Lemmatization Accuracy: 0.6432854186625791
Stemming Accuracy: 0.6447595324774861
without removing stop Words Accuracy: 0.6534776777160376


Trial

In [29]:
input= ['i will hit him with my bat','i got full mark','i am feeling depressed','hi im hari','tickets are sold out','you will loose marks','he is actually excited about milan']

for i in range(0,len(input)):
    words = nltk.word_tokenize(input[i])
    words = [stemmer.stem(word) for word in words]
    sentences = ' '.join(words)
    sentences = nfx.remove_stopwords(sentences)
    input[i] = sentences
print(input)
input_countvector = countvector.transform(input)
predictions = logreg.predict(input_countvector)
print(predictions)


['hit bat', 'got mark', 'feel depress', 'hi im hari', 'ticket sold', 'loos mark', 'actual excit milan']
['neutral' 'joy' 'sadness' 'neutral' 'neutral' 'joy' 'surprise']
